<a href="https://colab.research.google.com/github/KhinMyatNandar/Fraud-Detection/blob/main/CCFD_No_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
CCFD=pd.read_csv('/content/new_creditcard.csv')

In [ ]:
df=pd.DataFrame(CCFD)

In [ ]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,scaled_amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244200
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342584
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.158900
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.139886
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073813


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class',axis=1), df['Class'], test_size=0.30, random_state=42)

In [ ]:
print(y_train.value_counts)

<bound method IndexOpsMixin.value_counts of 9261      0
76107     0
240188    0
51280     0
30734     0
         ..
119879    0
259178    0
131932    0
146867    0
121958    0
Name: Class, Length: 198608, dtype: int64>


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

In [ ]:
results={}

In [ ]:
# Function to train and evaluate models without sampling technique
def train_and_evaluate_models(train_data, train_labels, test_data, test_labels):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'KNN': KNeighborsClassifier(),
        'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    }

    results_list = []

    for model_name, model in models.items():
        # Train the model
        model.fit(train_data, train_labels)

        # Training set predictions
        y_train_pred = model.predict(train_data)
        y_train_proba = model.predict_proba(train_data)[:, 1]

        # Testing set predictions
        y_test_pred = model.predict(test_data)
        y_test_proba = model.predict_proba(test_data)[:, 1]

        # Training set performance
        train_precision = precision_score(train_labels, y_train_pred)
        train_recall = recall_score(train_labels, y_train_pred)
        train_f1_score = f1_score(train_labels, y_train_pred)
        train_roc_auc = roc_auc_score(train_labels, y_train_proba)
        train_auprc = average_precision_score(train_labels, y_train_proba)

        # Testing set performance
        test_precision = precision_score(test_labels, y_test_pred)
        test_recall = recall_score(test_labels, y_test_pred)
        test_f1_score = f1_score(test_labels, y_test_pred)
        test_roc_auc = roc_auc_score(test_labels, y_test_proba)
        test_auprc = average_precision_score(test_labels, y_test_proba)

        results_list.append({
            'Model': model_name,
            'Train Precision': train_precision,
            'Train Recall': train_recall,
            'Train F1 Score': train_f1_score,
            'Train AUC-ROC': train_roc_auc,
            'Train AUPRC': train_auprc,
            'Test Precision': test_precision,
            'Test Recall': test_recall,
            'Test F1 Score': test_f1_score,
            'Test AUC-ROC': test_roc_auc,
            'Test AUPRC': test_auprc
        })

    return results_list

# Evaluate models on the original imbalanced dataset
all_results = train_and_evaluate_models(X_train, y_train, X_test, y_test)

# Print the results
for result in all_results:
    print(f"\nModel: {result['Model']}")
    print('Training set performance')
    print(f"- Precision : {result['Train Precision']:.4f}")
    print(f"- Recall : {result['Train Recall']:.4f}")
    print(f"- F1 Score : {result['Train F1 Score']:.4f}")
    print(f"- AUC-ROC : {result['Train AUC-ROC']:.4f}")
    print(f"- AUPRC : {result['Train AUPRC']:.4f}")

    print('Testing set performance')
    print(f"- Precision : {result['Test Precision']:.4f}")
    print(f"- Recall : {result['Test Recall']:.4f}")
    print(f"- F1 Score : {result['Test F1 Score']:.4f}")
    print(f"- AUC-ROC : {result['Test AUC-ROC']:.4f}")
    print(f"- AUPRC : {result['Test AUPRC']:.4f}")
    print('='*35)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:28:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: Logistic Regression
Training set performance
- Precision : 0.8678
- Recall : 0.6195
- F1 Score : 0.7229
- AUC-ROC : 0.9814
- AUPRC : 0.7633
Testing set performance
- Precision : 0.8605
- Recall : 0.5522
- F1 Score : 0.6727
- AUC-ROC : 0.9695
- AUPRC : 0.7106

Model: Random Forest
Training set performance
- Precision : 1.0000
- Recall : 1.0000
- F1 Score : 1.0000
- AUC-ROC : 1.0000
- AUPRC : 1.0000
Testing set performance
- Precision : 0.9519
- Recall : 0.7388
- F1 Score : 0.8319
- AUC-ROC : 0.9234
- AUPRC : 0.8078

Model: Decision Tree
Training set performance
- Precision : 1.0000
- Recall : 1.0000
- F1 Score : 1.0000
- AUC-ROC : 1.0000
- AUPRC : 1.0000
Testing set performance
- Precision : 0.7222
- Recall : 0.7761
- F1 Score : 0.7482
- AUC-ROC : 0.8878
- AUPRC : 0.5609

Model: KNN
Training set performance
- Precision : 0.9441
- Recall : 0.7965
- F1 Score : 0.8640
- AUC-ROC : 0.9999
- AUPRC : 0.9453
Testing set performance
- Precision : 0.9238
- Recall : 0.7239
- F1 Score : 0.8